# 2. Mengukur Kecepatan Object - OpenCV (lanjutan ...)

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import time

#### 2.0 previous function

In [2]:
def detectColor(img, lower, upper): 
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv.copy(), lower, upper)
        res = cv2.bitwise_and(img, img, mask= mask)
        return res

In [3]:
def findContourCircle(res): 
        x, y, r = 0, 0, 0
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        circle = []
        for cnt in contours:
            (x,y), radius = cv2.minEnclosingCircle(cnt)
            if radius > 10: 
                circle.append([int(x), int(y), int(radius)])
        if len(circle) > 0 :
            circle = np.array(circle)
            x = int(np.mean(circle[:, 0]))
            y = int(np.mean(circle[:, 1]))
            r = int(np.mean(circle[:, 2]))
        return x, y, r

In [4]:
def findCircelHoug(img):
    rs = []
    x, y, r = 0, 0, 0
    h, w, c = img.shape
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    edge = cv2.Canny(binary, 100, 200)
    binary = cv2.resize(binary, (0,0), fx=0.5, fy=0.5)
    
    cv2.imshow("binary", binary)
    
    circles = cv2.HoughCircles(edge, cv2.HOUGH_GRADIENT, 1, h/64, param1=150, param2=20, minRadius=50, maxRadius=100)
    if circles is not None:
        circles = np.uint16(np.around(circles))[0]
        x = int(np.mean(circles[:, 0]))
        y = int(np.mean(circles[:, 1]))
        rs.append(np.mean(circles[:, 2]))
        r = int(np.mean(rs))
    return x, y, r

In [5]:
def calc_speed(x, y, t):
    dx = float(x[1]) - float(x[0])
    dy = float(y[1]) - float(y[0])
    ds = math.sqrt( math.pow(dx,2) + math.pow(dy,2))
    dt = float(t[1]) - float(t[0])
    v = float(ds/dt)
    vx = float(dx/dt)
    vy = float(dy/dt)
    return v, vx, vy

In [6]:
color_ranges = dict(
    blue = dict(
        lower = np.array([110, 50, 50]),
        upper = np.array([130, 255, 255])
    ),
    orange = dict(
        lower = np.array([10, 50, 50]),
        upper = np.array([20, 255, 255])
    ) 
)

availabel_colors = color_ranges.keys()

#### 2.a Mengatur Referensi pengukuran kecepatan (x,y)

<img src="resource/circel.png" style="width:500px"></img>

In [7]:
dataset_dir = "dataset/"

filenames = os.listdir(dataset_dir)

print(filenames)

['video1.mp4', 'video2.mp4']


In [ ]:
color_input = input("pilih warna [blue/orange] :")

while color_input not in availabel_colors:
    color_input = input("%s tidak ditemukan, pilih warna [blue/orange] :" % color_input)

filename = filenames[0]
cap = cv2.VideoCapture(dataset_dir + filename)

t0, x0, y0 = time.time(), 0, 0
trajectory = [] 
xs, ys = [], [] 
scale = 10
i = 0 
while cap.isOpened():
    ret, img = cap.read()
    
    if ret:
        color = color_ranges[color_input]
        res = detectColor(img, color['lower'], color['upper'])
        
        x, y, r = findContourCircle(res)
        cv2.circle(img, (x, y), r, (255, 0, 255), 2)
         
        xs.append(x) 
        ys.append(y) 
  
        if i % 15 : 
            t = time.time()
            h, w, c = img.shape
            x_ = np.mean(xs[-60:]) 
            y_ = np.mean(h - np.array(ys[-60:])) 
            v, vx, vy = calc_speed([x0, x_], [y0, y_], [t0, t])
            print( "\rv : %7.2f, vx : %7.2f, vy : %7.2f \t\t" % (v, vx, vy), end="")

            t0, y0, x0 = t, y_, x_

            x, y = x - r, y + r
            cv2.putText(img, "(%d, %d)" % (x, y), (x + 15, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.line(img, (x,y), (x, y - r), (200, 200, 200), 2)
            cv2.line(img, (x,y), (x + r, y), (200, 200, 200), 2)
            cv2.arrowedLine(img, (x, y), (x, y - int(vy)*scale), (0,255,255), 2, cv2.LINE_AA) 
            cv2.arrowedLine(img, (x, y), (x - int(vx)*scale, y), (0,255,0), 2, cv2.LINE_AA) 
            cv2.arrowedLine(img, (x, y), (x - int(vx)*scale, y - int(vy)*scale), (0,0,255), 2, cv2.LINE_AA) 
            if v > 2 :
                cv2.putText(img, "%.2f" % v, (x - int(vx)*10, y - int(vy)*10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA) 
            trajectory.append([x,y]) 
        
        for pts in zip(trajectory, trajectory[1:]): 
            cv2.line(img, (pts[0][0], pts[0][1]), (pts[1][0], pts[1][1]), (255,255,0), 1, cv2.LINE_AA)
            
        if len(trajectory) > 60 :  
            vec_trajectory = np.array(trajectory)  
            xc = vec_trajectory[:,0]  
            yc = vec_trajectory[:,1]  
            orde = 2 if len(xc) <= 100 else 3 if len(xc) > 100 and len(xc) < 500 else 5 
            poly_solve = np.unique(yc), np.int0(np.poly1d(np.polyfit(yc, xc, orde))(np.unique(yc)))  
            cv2.polylines(img, [np.array(list(zip(poly_solve[1], poly_solve[0])))], False, (0,0,255), 1, cv2.LINE_AA)  
            
        #img = cv2.resize(img, (0,0), fx=0.8, fy=0.8)
        cv2.imshow(filename, img)
        
        del xs[:-120]
        del ys[:-120]
        i += 1 
        
        if cv2.waitKey(25) == ord('q'):
            break
    else :
        break
        
cap.release()
cv2.destroyAllWindows()

#### 2.b Mengubah satuan pixel ke jarak

$pixel/s * cm/pixel = cm/s$

In [ ]:
color_input = input("pilih warna [blue/orange] :")

while color_input not in availabel_colors:
    color_input = input("%s tidak ditemukan, pilih warna [blue/orange] :" % color_input)

filename = filenames[0]
cap = cv2.VideoCapture(dataset_dir + filename)

t0, x0, y0 = time.time(), 0, 0
trajectory = [] 
xs, ys = [], [] 
scale = 10
i = 0 
ratio = 40/480 # ratio dimensi : 30 cm / 700 pixel

while cap.isOpened():
    ret, img = cap.read()
    
    if ret:
        color = color_ranges[color_input]
        res = detectColor(img, color['lower'], color['upper'])
        
        x, y, r = findContourCircle(res)
        cv2.circle(img, (x, y), r, (255, 0, 255), 2)
        cv2.putText(img, "(%d, %d)" % (x, y), (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA) 
        xs.append(x) 
        ys.append(y) 
  
        if i % 15 : 
            t = time.time()
            h, w, c = img.shape
            x_ = np.mean(xs[-60:]) 
            y_ = np.mean(h - np.array(ys[-60:])) 
            v, vx, vy = calc_speed([x0, x_], [y0, y_], [t0, t])
            v_, vx_, vy_ = v*ratio, vx*ratio, vy*ratio
            print( "\rv : %7.2f cm/s, vx : %7.2f cm/s, vy : %7.2f cm/s \t\t" % (v_, vx_, vy_), end="")

            t0, y0, x0 = t, y_, x_

            x, y = x - r, y + r
            cv2.line(img, (x,y), (x, y - r), (200, 200, 200), 2)
            cv2.line(img, (x,y), (x + r, y), (200, 200, 200), 2)
            cv2.arrowedLine(img, (x, y), (x, y - int(vy_)*scale), (0,255,255), 2, cv2.LINE_AA) 
            cv2.arrowedLine(img, (x, y), (x - int(vx_)*scale, y), (0,255,0), 2, cv2.LINE_AA) 
            cv2.arrowedLine(img, (x, y), (x - int(vx_)*scale, y - int(vy)*scale), (0,0,255), 2, cv2.LINE_AA) 
            if v > 2 :
                cv2.putText(img, "%.2f cm/s" % v_, (x - int(vx)*10, y - int(vy)*10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA) 
            trajectory.append([x,y]) 
        
        for pts in zip(trajectory, trajectory[1:]): 
            cv2.line(img, (pts[0][0], pts[0][1]), (pts[1][0], pts[1][1]), (255,255,0), 1, cv2.LINE_AA)
            
        if len(trajectory) > 60 :  
            vec_trajectory = np.array(trajectory)  
            xc = vec_trajectory[:,0]  
            yc = vec_trajectory[:,1]  
            orde = 2 if len(xc) <= 100 else 3 if len(xc) > 100 and len(xc) < 500 else 5 
            poly_solve = np.unique(yc), np.int0(np.poly1d(np.polyfit(yc, xc, orde))(np.unique(yc)))  
            cv2.polylines(img, [np.array(list(zip(poly_solve[1], poly_solve[0])))], False, (0,0,255), 1, cv2.LINE_AA)  
            
        img = cv2.resize(img, (0,0), fx=0.8, fy=0.8)
        cv2.imshow(filename, img)
        
        del xs[:-120]
        del ys[:-120]
        i += 1 
        
        if cv2.waitKey(25) == ord('q'):
            break
    else :
        break
        
cap.release()
cv2.destroyAllWindows()

#### 2.c Add range bar

In [ ]:
color_input = input("pilih warna [blue/orange] :")

while color_input not in availabel_colors:
    color_input = input("%s tidak ditemukan, pilih warna [blue/orange] :" % color_input)

filename = filenames[0]
cap = cv2.VideoCapture(dataset_dir + filename)

t0, x0, y0 = time.time(), 0, 0
trajectory = [] 
xs, ys = [], [] 
scale = 10
i = 0 
ratio = 30/700 # ratio dimensi : 30 cm / 700 pixel

while cap.isOpened():
    ret, img = cap.read()
    
    if ret:
        h, w, c = img.shape
        color = color_ranges[color_input]
        res = detectColor(img, color['lower'], color['upper'])
        
        x, y, r = findContourCircle(res)
        cv2.circle(img, (x, y), r, (255, 0, 255), 2)
        cv2.putText(img, "(%d, %d)" % (x, y), (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA) 
        xs.append(x) 
        ys.append(y) 
  
        if i % 15 : 
            t = time.time()
            x_ = np.mean(xs[-60:]) 
            y_ = np.mean(h - np.array(ys[-60:])) 
            v, vx, vy = calc_speed([x0, x_], [y0, y_], [t0, t])
            v_, vx_, vy_ = v*ratio, vx*ratio, vy*ratio
            print( "\rv : %7.2f cm/s, vx : %7.2f cm/s, vy : %7.2f cm/s \t\t" % (v_, vx_, vy_), end="")

            t0, y0, x0 = t, y_, x_

            x, y = x - r, y + r
            cv2.line(img, (x,y), (x, y - r), (200, 200, 200), 2)
            cv2.line(img, (x,y), (x + r, y), (200, 200, 200), 2)
            cv2.arrowedLine(img, (x, y), (x, y - int(vy)*scale), (0,255,255), 2, cv2.LINE_AA) 
            cv2.arrowedLine(img, (x, y), (x - int(vx)*scale, y), (0,255,0), 2, cv2.LINE_AA) 
            cv2.arrowedLine(img, (x, y), (x - int(vx)*scale, y - int(vy)*scale), (0,0,255), 2, cv2.LINE_AA) 
            if v > 2 :
                cv2.putText(img, "%.2f cm/s" % v_, (x - int(vx)*10, y - int(vy)*10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA) 
            trajectory.append([x,y]) 
        
        for pts in zip(trajectory, trajectory[1:]): 
            cv2.line(img, (pts[0][0], pts[0][1]), (pts[1][0], pts[1][1]), (255,255,0), 1, cv2.LINE_AA)
            
        if len(trajectory) > 60 :  
            vec_trajectory = np.array(trajectory)  
            xc = vec_trajectory[:,0]  
            yc = vec_trajectory[:,1]  
            orde = 2 if len(xc) <= 100 else 3 if len(xc) > 100 and len(xc) < 500 else 5 
            poly_solve = np.unique(yc), np.int0(np.poly1d(np.polyfit(yc, xc, orde))(np.unique(yc)))  
            cv2.polylines(img, [np.array(list(zip(poly_solve[1], poly_solve[0])))], False, (0,0,255), 1, cv2.LINE_AA)  
            
        
        # add range bar
        (x1, y1), (x2, y2) = (int(w*0.3), int(h*0.8)), (int(w*0.3), int(h*0.9))
        (xm, ym) = (int(w*0.31), int(h*0.85 + 10))
        cv2.line(img, (x1, y1), (x2, y2), (200), 2)
        cv2.putText(img, "%.2f cm" % ((y2-y1)*ratio) , (xm, ym), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
        
        img = cv2.resize(img, (0,0), fx=0.8, fy=0.8)
        cv2.imshow(filename, img)
        
        del xs[:-120]
        del ys[:-120]
        i += 1 
        
        if cv2.waitKey(25) == ord('q'):
            break
    else :
        break
        
cap.release()
cv2.destroyAllWindows()

___
___

# 3. Basic GUI (desktop) Programming using pySimpleGUI

#### 3.1 Install library pySimple GUI

In [ ]:
! python -m pip install pysimplegui

In [8]:
import os
import PySimpleGUI as sg 

___
# Basic UI Element

#### 3.2 Create & Display Window

In [ ]:
window = sg.Window(title="Hello World", layout=[[]], margins=(200, 100))
window.read(timeout=3000)
window.close()

- Result : <br>
<img src="resource/basic-ui.png" style="width:400px"></img>
- One of the basic building blocks of PySimpleGUI is the `Window()`,
- on the created window, we can add `title` and a `layout` and set the `margins`,
- `margin` is margin for created layout `(margin-x, margin-y)`.

#### 3.3 Add text and button into `layout`

In [ ]:
layout = [[sg.Text("Hello from PySimpleGUI"), sg.Button("OK")], 
          [sg.Text("line 2")],
          [sg.Text("line 3")]]

window = sg.Window(title="Hello World", layout=layout, margins=(200, 100))

event, values = window.read()
print(event, values)

window.close()

- Result :<br>
<img src="resource/text-n-button.png" style="width:500px"></img>
- PySimpleGUI uses nested Python lists to lay out its elements. 
- In this case, you add a `Text()` element and a `Button()` element.

#### 3.4 Button Handler

In [ ]:
layout = [[sg.Text("klik button untuk close window!")], 
          [sg.Button("close")]]

window = sg.Window(title="Hello World", layout=layout, margins=(200, 100))

event, values = window.read()
if event == 'close':
    print('button close, clicked!')

window.close()

In [ ]:
layout = [[sg.Text("klik button untuk close window!")], 
          [sg.Button("print")]]

window = sg.Window(title="Hello World", layout=layout, margins=(200, 100))

while True:
    event, values = window.read()
    if event == 'print':
        print('print, Hello World!')
        
    if event == sg.WIN_CLOSED:
        break
        
window.close()

# Image Widget

#### 3.5 Simple Image Viewer

In [ ]:
image_viewer_layout= [
    [sg.Text("lena.png")],
    [sg.Image(filename="lena.png")],
]

window = sg.Window(title="Image Viewer", layout=image_viewer_layout, margins=(30, 30))
window.read()
window.close()

- result : <br>
<img src="resource/image-viewer.png" style="width:300px"></img>
- `Image()` :Image Element - show an image in the window for given `filename`. Should be a `GIF` or a `PNG` only.

#### 3.6 Image Viewer PySimpleGUI + OpenCV
- convert matrix image ke png menggunakan `cv2.imencode()`
- convert matrix `png` menjadi raw-data (bytes) menggunakan numpy function `.tobytes()`
- menggunakan `window[key].update(data=image_byte)` untuk mengupdate data pada widget dengan `key` tersebut.

In [ ]:
def readImage(filename = "lena.png"):
    img = cv2.imread(filename) 
    ret, frame_png = cv2.imencode('.png', img)
    img_byte = frame_png.tobytes()
    return img_byte

In [ ]:
image_viewer_layout= [
    [sg.Text("lena.png")],
    [sg.Image(filename='', key='-image-')],
]

window = sg.Window(title="Image Viewer", finalize=True, layout=image_viewer_layout, margins=(10, 10))

img_byte = readImage(filename = "lena.jpg")
window['-image-'].update(data=img_byte)

window.read()
window.close()

- `finalize=True` digunakan untuk dapat meng-update content widget dengan `key` yang belum di `.read()`

#### 3.7 Video Viewer + OpenCV

In [9]:
def readBytesFromFrame(cap):
    frame_byte = None
    ret, frame = cap.read()
    if ret :
        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        ret, frame_png = cv2.imencode('.png', frame)
        frame_byte = frame_png.tobytes()
    return ret, frame_byte

In [ ]:
video_viewer_layout = [[sg.Image(filename='', key='-image-')]]

window = sg.Window(title='Video Player', layout=video_viewer_layout, margins=(10, 10)) 

cap = cv2.VideoCapture("dataset/video1.mp4")

while cap.isOpened():   

    event, values = window.read(timeout=25)  
        
    ret, frame_byte = readBytesFromFrame(cap)
    
    if not ret or event == sg.WIN_CLOSED:
        break
        
    window['-image-'].update(data=frame_byte)

cap.release()
window.close()

- `sg.WIN_CLOSED` adalah event close window by user 
- `window.read(timeout=20)` akan membaca semua data yang dibutuhkan widget/element dan menampilkanya.
- `timeout=20` proses pembacaat akan dilakukan selama `20ms`

#### 3.7 Video Viewer with Close Button

In [ ]:
video_viewer_layout = [[sg.Image(filename='', key='-image-')], 
                       [sg.Button('close')]]

window = sg.Window(title='Video Player', layout=video_viewer_layout, margins=(10, 10)) 

cap = cv2.VideoCapture("dataset/video1.mp4")

while cap.isOpened():   

    event, values = window.read(timeout=20)  
        
    ret, frame_byte = readBytesFromFrame(cap)
    
    if not ret or event == 'close' or event == sg.WIN_CLOSED:
        break
        
    window['-image-'].update(data=frame_byte)

cap.release()
window.close()

___
## Widget Browse File

#### 3.8 Browse Image & View
- `sg.FileBrowse` digunakan untuk memunculkan tombol browse file,
- `file_types=(("Image", "*.png"),)` akan membatasi file yang di browse adalah gambar dengan extension `.png`
- `sg.In` adalah widget input text untuk menampung `path` lokasi gambar (e.g : C:/Users/Jhon/image.png)
- `size=(25, 1)` adalah 25 char 1 baris pada input text `sg.In`
- `enable_events=True` agar widgetdapat menghandle event

In [ ]:
layout = [[ 
            sg.Text("Image File"),
            sg.In(size=(25, 1), enable_events=True, key="-filename-"),
            sg.FileBrowse(file_types=((".jpg", "*.png"),)) 
          ],
          [ 
            sg.Image(filename='', key='-image-') 
          ]]

window = sg.Window(title="Image Browse & Viewer", finalize=True, layout=layout, margins=(10, 10))

while True :
    event, values = window.read()
    print(event, values)
    if event == sg.WIN_CLOSED :
        break
    if event == '-filename-' :
        filename = values['-filename-']
        img_byte = readImage(filename = filename)
        window['-image-'].update(data=img_byte)

window.close()

#### 3.9 Browse Image & View with placeholder empty image

In [ ]:
layout = [[ 
            sg.Text("Image File"),
            sg.In(size=(25, 1), enable_events=True, key="-filename-"),
            sg.FileBrowse(file_types=(("Image", "*.png"),)) 
          ],
          [ 
            sg.Image(filename='empty.png', key='-image-') 
          ]]

window = sg.Window(title="Image Browse & Viewer V2", finalize=True, layout=layout, margins=(10, 10))

while True :
    event, values = window.read()
    if event == sg.WIN_CLOSED :
        break
    if event == '-filename-' :
        filename = values['-filename-']
        img_byte = readImage(filename = filename)
        window['-image-'].update(data=img_byte)

window.close()

#### 3.10 Browse & Play Video

In [ ]:
layout = [[ 
            sg.Text("Video File"),
            sg.In(size=(25, 1), enable_events=True, key="-filename-"),
            sg.FileBrowse(file_types=(("Video", "*.mp4"),)) 
          ],
          [ 
            sg.Image(filename='empty.png', key='-image-') 
          ]]

window = sg.Window(title="Video Browse & Viewer", finalize=True, layout=layout, margins=(10, 10))

cap = None
filename = ''
while True :
    event, values = window.read(timeout=25)
    if event == sg.WIN_CLOSED :
        break
    if event == '-filename-' :
        filename = values['-filename-']
        cap = cv2.VideoCapture(filename)
        
    if cap is not None :
        ret, frame_byte = readBytesFromFrame(cap)
        if not ret:
            cap.release()
            break
        window['-image-'].update(data=frame_byte)

window.close()

#### 3.11 Add Play and Pause Button

In [ ]:
layout = [[ 
            sg.Text("Video File"),
            sg.In(size=(25, 1), enable_events=True, key="-filename-"),
            sg.FileBrowse(file_types=(("Video", "*.mp4"),)),
            sg.Button('play'),
            sg.Button('pause')
          ],
          [ 
            sg.Image(filename='empty.png', key='-image-') 
          ]]

window = sg.Window(title="Video Browse & Viewer", finalize=True, layout=layout, margins=(10, 10))

cap = None
stat = ''
filename = ''
while True :
    event, values = window.read(timeout=25)
    if event == sg.WIN_CLOSED :
        break
    if event == '-filename-' :
        filename = values['-filename-']
        cap = cv2.VideoCapture(filename)
        
    if event == 'play' or event == 'pause':
        stat = event

    if stat == 'play':
        if cap is None:
            cap = cv2.VideoCapture(filename)
        ret, frame_byte = readBytesFromFrame(cap)
        if not ret:
            cap.release()
            cap = None
            break
        window['-image-'].update(data=frame_byte)
        
window.close()

#### 3.12 Add button Stop video

In [ ]:
layout = [[ 
            sg.Text("Video File"),
            sg.In(size=(25, 1), enable_events=True, key="-filename-"),
            sg.FileBrowse(file_types=(("Video", "*.mp4"),)),
            sg.Button('play'),
            sg.Button('pause'),
            sg.Button('stop')
          ],
          [ 
            sg.Image(filename='empty.png', key='-image-') 
          ]]

window = sg.Window(title="Video Browse & Viewer", finalize=True, layout=layout, margins=(10, 10))

cap = None
stat = ''
filename = ''
while True :
    event, values = window.read(timeout=25)
    if event == sg.WIN_CLOSED :
        break
    if event == '-filename-' :
        filename = values['-filename-']
        
    if event == 'play' or event == 'pause' or event == 'stop':
        stat = event

    if stat == 'play':
        if cap is None:
            cap = cv2.VideoCapture(filename)
        ret, frame_byte = readBytesFromFrame(cap)
        if not ret:
            cap.release()
            cap = None
            break
        window['-image-'].update(data=frame_byte)
    
    if cap is not None and stat == 'stop':
        cap.release()
        cap = None
        window['-image-'].update(filename='empty.png')
        
window.close()

#### 3.14 Customized Button

In [ ]:
image_play = "asset/play.png"
image_pause = "asset/pause.png"
image_stop = "asset/stop.png"

layout = [[sg.Text("Hello from PySimpleGUI")], 
          [
          sg.Button(image_filename=image_play),
          sg.Button(image_filename=image_pause),
          sg.Button(image_filename=image_stop)
          ]]

window = sg.Window(title="Customized Button", layout=layout, margins=(200, 100))

event, values = window.read()
print(event, values)

window.close()

#### 3.15 Change button Color

In [ ]:
image_play = "asset/play.png"
image_pause = "asset/pause.png"
image_stop = "asset/stop.png"

layout = [[sg.Text("Hello from PySimpleGUI")], 
          [
          sg.Button(image_filename=image_play, button_color=('#FFF', '#FFF')),
          sg.Button(image_filename=image_pause, button_color=('#FFF', '#FFF')),
          sg.Button(image_filename=image_stop, button_color=('#FFF', '#FFF'))
          ]]

window = sg.Window(title="Customized Button", layout=layout, margins=(200, 100))

event, values = window.read()
print(event, values)

window.close()

- `button_color` : color of the buttons shown `(text color, button color)`

#### 3.16 Change button size 

In [ ]:
image_play = "asset/play.png"
image_pause = "asset/pause.png"
image_stop = "asset/stop.png"

layout = [[sg.Text("Customized Button")], 
          [
          sg.Button(image_filename=image_play, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0),
          sg.Button(image_filename=image_pause, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0),
          sg.Button(image_filename=image_stop, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0)
          ]]

window = sg.Window(title="Customized Button", layout=layout, margins=(200, 100))

event, values = window.read()
print(event, values)

window.close()

- `image_size` : Size of image in pixels `(width, height)`
- `image_subsample` : Amount to divide the size by.  2 means your image will be 1/2 the size.  3 means 1/3.
- `border_width` : border width on button (pixel).

### 3.17 Apply Customized Button to Video Player

In [23]:
image_play = "asset/play.png"
image_pause = "asset/pause.png"
image_stop = "asset/stop.png"

layout = [[ 
            sg.Text("Video File"),
            sg.In(size=(25, 1), enable_events=True, key="-filename-"),
            sg.FileBrowse(file_types=(("Video", "*.mp4"),)),
            sg.Button(key='play', image_filename=image_play, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0),
            sg.Button(key='pause', image_filename=image_pause, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0),
            sg.Button(key='stop', image_filename=image_stop, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0)
          ],
          [ 
            sg.Image(filename='empty.png', key='-image-') 
          ]]

window = sg.Window(title="Video Browse & Viewer", finalize=True, layout=layout, margins=(10, 10))

cap = None
stat = ''
filename = ''
while True :
    event, values = window.read(timeout=25)
    if event == sg.WIN_CLOSED :
        break
    if event == '-filename-' :
        filename = values['-filename-']
        
    if event == 'play' or event == 'pause' or event == 'stop':
        stat = event

    if stat == 'play':
        if cap is None:
            cap = cv2.VideoCapture(filename)
        ret, frame_byte = readBytesFromFrame(cap)
        if not ret:
            cap.release()
            break
        window['-image-'].update(data=frame_byte)
    
    if cap is not None and stat == 'stop':
        cap.release()
        cap = None
        window['-image-'].update(filename='empty.png')
        
window.close()

### 3.18 PySimpleGUI theme

- list availabel theme

In [10]:
sg.theme_previewer() 

- change theme

In [20]:
sg.theme('DarkTeal8')

'DarkTeal8'

In [21]:

layout = [[sg.Text("Hello from PySimpleGUI")], 
          [sg.Button("OK")]]

window = sg.Window(title="Hello World", layout=layout, margins=(200, 100))

event, values = window.read()
print(event, values)

window.close()

OK {}


### 3.19 Video Player + Play list

- UI mockup : <br>
<img src="resource/video-payer-playlist.png" style="width:700px"></img>

In [24]:
file_list_column = [
    [
        sg.Text("Video Folder"),
        sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
        sg.FolderBrowse(),
    ],
    [
        sg.Listbox(values=[], enable_events=True, size=(40, 20), key="-FILE LIST-")
    ],
]

image_viewer_column = [
    [sg.Image(filename='empty.png', key="-IMAGE-")],
]

layout = [
    [
        sg.Column(file_list_column),
        sg.VSeperator(),
        sg.Column(image_viewer_column),
    ]
]


In [26]:
os.listdir("asset/")

['.ipynb_checkpoints', 'pause.png', 'play.png', 'stop.png']

In [27]:
window = sg.Window("Video Viewer + Play List", layout=layout, margins=(10, 10))

cap = None
video_path = ''
while True:
    event, values = window.read(timeout=25)
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
        
    if event == "-FOLDER-":
        folder = values["-FOLDER-"]
        try:
            file_list = os.listdir(folder)
        except:
            file_list = []
            
        fnames = [
            f
            for f in file_list
            if os.path.isfile(os.path.join(folder, f))
            and f.lower().endswith(("video", ".mp4"))
        ]
        
        window["-FILE LIST-"].update(fnames)
    
    elif event == "-FILE LIST-":  
        try:
            folder = values["-FOLDER-"]
            filename = values["-FILE LIST-"][0]
            video_path = os.path.join(folder, filename)
            
            if cap is None :
                cap = cv2.VideoCapture(video_path)
        except:
            pass
        
    if cap is not None :
        ret, frame_byte = readBytesFromFrame(cap)
        if not ret :
            cap.release()
            cap = None
            break
        window["-IMAGE-"].update(data=frame_byte)
        
window.close()

#### 3.20 Membuat Video Player + Playlist + Control Button (play, pause, stop)

- UI mockup : <br>
<img src="resource/video-payer-playlist-2.png" style="width:700px"></img>

In [28]:
image_play = "asset/play.png"
image_pause = "asset/pause.png"
image_stop = "asset/stop.png"

file_list_column = [
    [
        sg.Text("Video Folder"),
        sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
        sg.FolderBrowse(),
    ],
    [
        sg.Listbox(values=[], enable_events=True, size=(40, 20), key="-FILE LIST-")
    ],
]

image_viewer_column = [
    [
    sg.Button(key='play', image_filename=image_play, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0),
    sg.Button(key='pause', image_filename=image_pause, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0),
    sg.Button(key='stop', image_filename=image_stop, button_color=('#FFF', '#FFF'), image_size=(40, 40), image_subsample=3, border_width=0)
    ],
    [sg.Image(filename='empty.png', key="-IMAGE-")],
]

layout = [
    [
        sg.Column(file_list_column),
        sg.VSeperator(),
        sg.Column(image_viewer_column),
    ]
]


In [29]:
window = sg.Window("Video Viewer + Play List", layout=layout, margins=(10, 10))

cap = None
video_path = ''
stat = ''
while True:
    event, values = window.read(timeout=0)
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
        
    if event == "-FOLDER-":
        folder = values["-FOLDER-"]
        try:
            file_list = os.listdir(folder)
        except:
            file_list = []
            
        fnames = [
            f
            for f in file_list
            if os.path.isfile(os.path.join(folder, f))
            and f.lower().endswith(("video", ".mp4"))
        ]
        
        window["-FILE LIST-"].update(fnames)
    
    elif event == "-FILE LIST-":  
        try:
            folder = values["-FOLDER-"]
            filename = values["-FILE LIST-"][0]
            video_path = os.path.join(folder, filename)

        except:
            pass
        
    if event == 'play' or event == 'pause' or event == 'stop':
        stat = event
        
    if stat == 'play':
        if cap is None:
            cap = cv2.VideoCapture(video_path)
        ret, frame_byte = readBytesFromFrame(cap)
        if not ret:
            cap.release()
            break
        window['-IMAGE-'].update(data=frame_byte)
    
    if cap is not None and stat == 'stop':
        cap.release()
        cap = None
        window['-IMAGE-'].update(filename='empty.png') 

        
window.close()

# Sumber :

- https://pysimplegui.readthedocs.io/en/latest/cookbook/